***Leaf Classification***

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
dataframe = pd.read_csv("/content/leaves.csv")

In [ ]:
dataframe.head()

,Unnamed: 0,area,perimeter,length,width,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stdd_r,stdd_g,std_b,contrast,correlation,inverse_difference_moments,entropy,target
0,0,51847.5,3684.821951,1423,970,1.467010,26.622499,261.881726,4.597799,5.983273,4.049544,26.751711,32.143155,25.739416,4.580835,0.995637,0.967845,0.705123,16
1,0,49659.5,3487.660474,1382,872,1.584862,24.267341,244.943577,4.606244,5.977342,3.862396,26.724926,32.854526,24.470043,3.940764,0.995079,0.969464,0.660537,16
2,0,909247.5,4318.165981,1528,1060,1.441509,1.781341,20.507681,165.169551,203.266355,151.322441,95.072479,54.146559,107.461724,64.121415,0.993846,0.583829,7.045166,15
3,0,875272.0,4266.859409,1495,1094,1.366545,1.868596,20.800493,31.015689,68.073624,16.323695,47.430969,76.688455,31.684781,88.173861,0.992488,0.590544,6.972287,15
4,0,52925.5,3632.135654,1423,946,1.504228,25.434960,249.263765,5.049740,6.541959,4.179818,28.468008,34.676257,26.519846,5.930811,0.993337,0.966138,0.731323,16


In [ ]:
dataframe = dataframe.iloc[:,1:]
dataframe.head()


,area,perimeter,length,width,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stdd_r,stdd_g,std_b,contrast,correlation,inverse_difference_moments,entropy,target
0,51847.5,3684.821951,1423,970,1.467010,26.622499,261.881726,4.597799,5.983273,4.049544,26.751711,32.143155,25.739416,4.580835,0.995637,0.967845,0.705123,16
1,49659.5,3487.660474,1382,872,1.584862,24.267341,244.943577,4.606244,5.977342,3.862396,26.724926,32.854526,24.470043,3.940764,0.995079,0.969464,0.660537,16
2,909247.5,4318.165981,1528,1060,1.441509,1.781341,20.507681,165.169551,203.266355,151.322441,95.072479,54.146559,107.461724,64.121415,0.993846,0.583829,7.045166,15
3,875272.0,4266.859409,1495,1094,1.366545,1.868596,20.800493,31.015689,68.073624,16.323695,47.430969,76.688455,31.684781,88.173861,0.992488,0.590544,6.972287,15
4,52925.5,3632.135654,1423,946,1.504228,25.434960,249.263765,5.049740,6.541959,4.179818,28.468008,34.676257,26.519846,5.930811,0.993337,0.966138,0.731323,16


In [ ]:
X = dataframe.drop(['target'],axis='columns')
X.head()

,area,perimeter,length,width,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stdd_r,stdd_g,std_b,contrast,correlation,inverse_difference_moments,entropy
0,51847.5,3684.821951,1423,970,1.467010,26.622499,261.881726,4.597799,5.983273,4.049544,26.751711,32.143155,25.739416,4.580835,0.995637,0.967845,0.705123
1,49659.5,3487.660474,1382,872,1.584862,24.267341,244.943577,4.606244,5.977342,3.862396,26.724926,32.854526,24.470043,3.940764,0.995079,0.969464,0.660537
2,909247.5,4318.165981,1528,1060,1.441509,1.781341,20.507681,165.169551,203.266355,151.322441,95.072479,54.146559,107.461724,64.121415,0.993846,0.583829,7.045166
3,875272.0,4266.859409,1495,1094,1.366545,1.868596,20.800493,31.015689,68.073624,16.323695,47.430969,76.688455,31.684781,88.173861,0.992488,0.590544,6.972287
4,52925.5,3632.135654,1423,946,1.504228,25.434960,249.263765,5.049740,6.541959,4.179818,28.468008,34.676257,26.519846,5.930811,0.993337,0.966138,0.731323


In [ ]:
y = dataframe.target
y.head()

0    16
1    16
2    15
3    15
4    16
Name: target, dtype: int64

***Train-test-split***

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 142)

***Feature Scaling***

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
def best_model_selection(X,y):
     algorithms={
         'random forest':{
             'model':RandomForestClassifier(),
             'params':{
                 'n_estimators':[5,10,15,20],
                 'criterion':['entropy'],
                 "max_features": ['sqrt'],
                 "max_depth": [10, 15, 20, 25, 30],
                 "min_samples_split": [2, 4, 6],
                 "min_samples_leaf": [1, 2],
                 "bootstrap": [False],
             }
         },
       'svm':{
             'model': SVC(),
             'params':[{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
               'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
             ]
         },
        'KNN':{
            'model':KNeighborsClassifier(),
            'params':{
                'n_neighbors':list(range(1, 31))
            }
        },
        'naive bayes':{
            'model':  GaussianNB(),
            'params':{
                'var_smoothing': np.logspace(0,-9, num=100)
            }
        }
     }
     scores=[]
     for algo_name, config in algorithms.items():
        gs =  GridSearchCV(config['model'], config['params'],cv=5)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
     return pd.DataFrame(scores,columns=['model','best_score','best_params'])


In [ ]:
best_model_selection(X_train,y_train)

,model,best_score,best_params
0,random forest,0.851587,"{'bootstrap': False, 'criterion': 'entropy', '..."
1,svm,0.900296,"{'C': 100, 'kernel': 'linear'}"
2,KNN,0.832829,{'n_neighbors': 1}
3,naive bayes,0.736856,{'var_smoothing': 0.0001232846739442066}


***From the table we found that SVM has best score. Hence, SVM will be applied.***

In [ ]:
from sklearn import svm

In [ ]:
params = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
               'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
             ]

In [ ]:
from sklearn.model_selection import GridSearchCV
svm_c = GridSearchCV(svm.SVC(decision_function_shape='ovr'), params, cv=5)
svm_c.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

In [ ]:
y_pred = svm_c.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

0.8987783595113438


***Export the tested model to a pickle file***

In [ ]:
 import pickle
 with open('Leaf_classification_model.pickle','wb') as f:
     pickle.dump(svm_c,f)